#### Laboratorio 4 : Optimizacion

Jorge Caballeros 20009

Alejandra Guzmán 20262

__Instrucciones:__
Utilice el dataset compartido en el espacio para construir la mejor red neuronal posible según las técnicas de
optimización vistas en clase.
__Objetivo:__  
Maximizar accuracy del pronóstico de la columna Y. default payment next month 1.


In [48]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))


[]


In [49]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Cargar el dataset
data = pd.read_excel("data.xlsx")

# Establecer la primera fila como encabezados de columnas
data.columns = data.iloc[0]
data = data.drop(0)

# Eliminar la columna de identificación
data = data.drop(columns=["ID"])

# Convertir las columnas a tipos numéricos
data = data.apply(pd.to_numeric)

# Separar características y etiquetas
X = data.drop(columns=["default payment next month"])
y = data["default payment next month"].values

# Dividir el dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar las características
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train.shape, X_test.shape, y_train.shape, y_test.shape


((24000, 23), (6000, 23), (24000,), (6000,))

Random models 

In [50]:
from tensorflow import keras
import random

# Definir hiperparámetros aleatorios
def random_hyperparameters():
    num_layers = random.randint(1, 5)
    neurons_per_layer = random.choice([16, 32, 64, 128])
    activation_function = random.choice(['relu', 'tanh', 'sigmoid', 'softmax', 'elu', 'selu'])
    loss_function = 'binary_crossentropy'
    optimizer = random.choice(['sgd', 'adam', 'rmsprop'])
    regularizer = random.choice(['dropout', 'l1', 'l2'])
    batch_size = random.choice([16, 32, 64, 128])
    
    return num_layers, neurons_per_layer, activation_function, loss_function, optimizer, regularizer, batch_size

def create_model(num_layers, neurons_per_layer, activation_function, loss_function, optimizer, regularizer, batch_size):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=(X_train.shape[1],)))
    
    for _ in range(num_layers):
        if regularizer == 'dropout':
            model.add(keras.layers.Dropout(0.5))
        elif regularizer == 'l1':
            model.add(keras.layers.Dense(neurons_per_layer, activation=activation_function, kernel_regularizer=keras.regularizers.l1(0.01)))
        elif regularizer == 'l2':
            model.add(keras.layers.Dense(neurons_per_layer, activation=activation_function, kernel_regularizer=keras.regularizers.l2(0.01)))
        else:
            model.add(keras.layers.Dense(neurons_per_layer, activation=activation_function))
            
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])
    
    return model

num_layers, neurons_per_layer, activation_function, loss_function, optimizer, regularizer, batch_size = random_hyperparameters()
# Crear un modelo de prueba para verificar la función
model = create_model(num_layers, neurons_per_layer, activation_function, loss_function, optimizer, regularizer, batch_size)
model.summary()


Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_62 (Dense)            (None, 16)                384       


                                                                 
 dense_63 (Dense)            (None, 1)                 17        
                                                                 
Total params: 401
Trainable params: 401
Non-trainable params: 0
_________________________________________________________________


In [51]:
from sklearn.model_selection import StratifiedKFold
import numpy as np

def fit_solution(model, X, y):
    """
    Entrena un modelo y devuelve su rendimiento usando validación cruzada estratificada.
    """
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []
    
    for train_idx, val_idx in kfold.split(X, y):
        train_data, val_data = X[train_idx], X[val_idx]
        train_labels, val_labels = y[train_idx], y[val_idx]
        
        model.fit(train_data, train_labels, epochs=50, verbose=0, batch_size=32, validation_data=(val_data, val_labels))
        _, accuracy = model.evaluate(val_data, val_labels, verbose=0)
        scores.append(accuracy)
    
    return np.mean(scores)


# Definir select_fitness_solution para seleccionar los modelos más aptos
def select_fitness_solution(population, fitnesses, top_k=3):
    """
    Selecciona los modelos más aptos.
    """
    # Ordenar la población por rendimiento
    sorted_indices = np.argsort(fitnesses)[::-1]
    return [population[i] for i in sorted_indices[:top_k]]

# Definir crossover para combinar dos soluciones
def crossover(parent1, parent2):
    """
    Combina dos soluciones para crear una nueva.
    """
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

# Definir mutation para hacer pequeños ajustes en una solución
def mutate(solution):
    """
    Hace pequeños ajustes en una solución para explorar el espacio de hiperparámetros.
    """
    mutation_point = random.randint(0, len(solution) - 1)
    if mutation_point == 0:
        solution[mutation_point] = random.randint(1, 5)
    elif mutation_point == 1:
        solution[mutation_point] = random.choice([16, 32, 64, 128])
    elif mutation_point == 2:
        solution[mutation_point] = random.choice(['relu', 'tanh', 'sigmoid', 'softmax', 'elu', 'selu'])
    elif mutation_point == 3:
        solution[mutation_point] = 'binary_crossentropy'
    elif mutation_point == 4:
        solution[mutation_point] = random.choice(['sgd', 'adam', 'rmsprop'])
    elif mutation_point == 5:
        solution[mutation_point] = random.choice(['dropout', 'l1', 'l2'])
    elif mutation_point == 6:
        solution[mutation_point] = random.choice([16, 32, 64, 128])
    return solution


In [52]:
import numpy as np
# Variables globales
NUM_GENERATIONS = 10
POPULATION_SIZE = 20
TOP_K = 3

# Inicializar la población con modelos aleatorios
population = [random_hyperparameters() for _ in range(POPULATION_SIZE)]

# Iterar a través de las generaciones
for generation in range(NUM_GENERATIONS):
    print('--------------------------------------------------')
    print(f"Generación {generation + 1} de {NUM_GENERATIONS}")
    
    # Evaluar la aptitud de la población
    fitnesses = []
    for i, hyperparameters in enumerate(population):
        print(f"Evaluando modelo {i + 1} con hiperparámetros: {hyperparameters}")
        model = create_model(*hyperparameters)
        fitness = fit_solution(model, X_train, y_train)
        fitnesses.append(fitness)
        print(f"Accuracy del modelo {i + 1}: {fitness:.4f}")
    
    # Seleccionar las mejores soluciones
    best_solutions = select_fitness_solution(population, fitnesses, TOP_K)
    
    # Mostrar las mejores soluciones
    for i, solution in enumerate(best_solutions, 1):
        print(f"Top {i} solución: {solution} con accuracy: {fitnesses[population.index(solution)]:.4f}")
    
    # Generar una nueva población
    new_population = []
    for i in range(POPULATION_SIZE // 2):
        parent1 = random.choice(best_solutions)
        parent2 = random.choice(best_solutions)
        while parent1 == parent2:
            parent2 = random.choice(best_solutions)
        
        child1, child2 = crossover(parent1, parent2)
        
        # Mutación con una probabilidad del 10%
        if random.random() < 0.1:
            child1 = mutate(child1)
        if random.random() < 0.1:
            child2 = mutate(child2)
        
        new_population.extend([child1, child2])
    
    # Actualizar la población para la siguiente generación
    population = new_population

# Seleccionar la mejor solución al final de todas las generaciones
final_best_index = np.argmax(fitnesses)
print(f"Mejor solución final: {population[final_best_index]} con accuracy: {fitnesses[final_best_index]:.4f}")


--------------------------------------------------
Generación 1 de 10
Evaluando modelo 1 con hiperparámetros: (5, 16, 'sigmoid', 'binary_crossentropy', 'rmsprop', 'l2', 64)
Accuracy del modelo 1: 0.7782
Evaluando modelo 2 con hiperparámetros: (3, 128, 'selu', 'binary_crossentropy', 'rmsprop', 'l1', 32)
Accuracy del modelo 2: 0.7782
Evaluando modelo 3 con hiperparámetros: (2, 32, 'selu', 'binary_crossentropy', 'rmsprop', 'dropout', 16)
Accuracy del modelo 3: 0.7825
Evaluando modelo 4 con hiperparámetros: (4, 64, 'tanh', 'binary_crossentropy', 'adam', 'l2', 32)
Accuracy del modelo 4: 0.8171
Evaluando modelo 5 con hiperparámetros: (1, 32, 'softmax', 'binary_crossentropy', 'sgd', 'l1', 128)
Accuracy del modelo 5: 0.7782
Evaluando modelo 6 con hiperparámetros: (4, 128, 'selu', 'binary_crossentropy', 'adam', 'l2', 16)
Accuracy del modelo 6: 0.8132
Evaluando modelo 7 con hiperparámetros: (3, 64, 'elu', 'binary_crossentropy', 'adam', 'dropout', 16)
Accuracy del modelo 7: 0.7793
Evaluando model

TypeError: 'tuple' object does not support item assignment